In [1]:
import numpy as np
import requests
import matplotlib.pyplot as plt
import ast
import logging
import os
# import skimage
from PIL import Image
import gradio as gr

In [2]:


def make_prediction(img:np.array, img_size:int, host:str)-> requests:
  """
  Make a binary prediction from a single fingerprint image.
  Args:
   img - The request image array.
   img_size - The image height and width.
   host - The hostname[:port] of the model service.
  Returns: The model output prediction.
  """
  #
  # Build the request payload. The "name" must match the
  # input layer name of the model.
  #
  req2 = {
      "inputs": [
        {
          "name": "conv2d_12_input",
          "shape": [1, img_size, img_size, 1],
          "datatype": "FP32",
          "data": img.tolist()
        }
      ]
    }

  url = f'https://{host}/v2/models/fingerprint/infer'
  r = requests.post(url, json=req2)
  return r



In [3]:
def predict(image):
    # return image.rotate(45)
    logging.info(f"predict(): type = {type(image)}")
    logging.info(f"predict(): image = {image}")

    resized = image.resize((96, 96)).convert('L')
    logging.info(f"predict(): resized = {resized}")
    np_image = np.asarray(resized)
    host = "triton-models.apps.ocp.sandbox2000.opentlc.com"
    r = make_prediction(np_image, 96, host)
    p = ast.literal_eval(r.content.decode())
    return f"Prediction = {p}"


demo = gr.Interface(predict, gr.Image(type="pil"), "text",
    flagging_options=["blurry", "incorrect", "other"], examples=[
        os.path.join(os.path.abspath(''), "images/103__F_Left_index_finger.png"),
        os.path.join(os.path.abspath(''), "images/275__F_Left_index_finger.png"),
        os.path.join(os.path.abspath(''), "images/232__M_Right_index_finger.png"),
        os.path.join(os.path.abspath(''), "images/504__M_Right_index_finger.png")
        ])

In [6]:
if __name__ == "__main__":

  logging.basicConfig(level=logging.INFO)

  demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


INFO:root:predict(): type = <class 'PIL.Image.Image'>
INFO:root:predict(): image = <PIL.Image.Image image mode=RGB size=96x103 at 0x12459EE80>
INFO:root:predict(): resized = <PIL.Image.Image image mode=L size=96x96 at 0x1245AD7C0>
INFO:root:predict(): type = <class 'PIL.Image.Image'>
INFO:root:predict(): image = <PIL.Image.Image image mode=RGB size=2020x1244 at 0x1245AD3A0>
INFO:root:predict(): resized = <PIL.Image.Image image mode=L size=96x96 at 0x12454BD90>
INFO:root:predict(): type = <class 'PIL.Image.Image'>
INFO:root:predict(): image = <PIL.Image.Image image mode=RGB size=96x103 at 0x1245BB310>
INFO:root:predict(): resized = <PIL.Image.Image image mode=L size=96x96 at 0x1245BBCA0>
INFO:root:predict(): type = <class 'PIL.Image.Image'>
INFO:root:predict(): image = <PIL.Image.Image image mode=RGB size=96x103 at 0x1245BBA00>
INFO:root:predict(): resized = <PIL.Image.Image image mode=L size=96x96 at 0x1245BB6A0>
INFO:root:predict(): type = <class 'PIL.Image.Image'>
INFO:root:predict()

In [5]:
demo.close()

Closing server running on port: 7860


In [ ]:
#
# This cell will be deleted and should not be run.
#

#
# Check that $HOST is set.
#
try:
assert os.getenv('HOST')
logging.info(f"HOST = {os.getenv('HOST')}")
except:
logging.error("HOST environment variable is not set!")
exit()

host = os.getenv('HOST')
url = f'https://{host}/v2'
r = ""

try:
r = requests.get(url)
# logging.info("")
# logging.info(f'Triton Server Status:')
# logging.info("")
# logging.info(f'{r.content.decode()}')
# logging.info("")
except:
logging.error(f"Requests Error! {r}")
exit()

#
# Make 4 single image predictions.
#
img_size = 96
filepath = 'images'
filenames = [
'103__F_Left_index_finger.png',
'275__F_Left_index_finger.png',
'232__M_Right_index_finger.png',
'504__M_Right_index_finger.png'
]

for filename in filenames:

#
# Read in an image file using skimage.
#
f = f'{filepath}/{filename}'
F_Left_index = skimage.io.imread(f, as_gray=True)
img_resized = skimage.transform.resize(F_Left_index, (96, 96))

logging.debug(f'img_resized.shape = {img_resized.shape}')
logging.info(f'skimage type: {type(img_resized)}')

#
# Make an inference via REST.
#
try:
  r = make_prediction(img_resized, img_size, host)
  logging.debug(f'REST inference response = {r}')
  p = ast.literal_eval(r.content.decode())
  logging.info(f"Fingerprint = {filename}, Prediction = {p['outputs'][0]['data']}")
  f = plt.figure()
  f.set_figwidth(1)
  f.set_figheight(1)  
  imgplot = plt.imshow(img_resized, cmap="gray") 
  plt.show()
except:
  logging.error(f"Requests POST Error {r.content}")
